In [391]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [392]:
#数据预处理
data1=pd.read_csv('../data/比赛训练集.csv',encoding='gbk')
data2=pd.read_csv('../data/糖尿病遗传风险检测挑战赛测试集B.csv',encoding='utf-8')

In [393]:
#label标记为-1
data2['患有糖尿病标识']=-1

In [394]:
#训练集和测试机合并
data=pd.concat([data1,data2],axis=0,ignore_index=True)
data

,编号,性别,出生年份,体重指数,糖尿病家族史,舒张压,口服耐糖量测试,胰岛素释放实验,肱三头肌皮褶厚度,患有糖尿病标识
0,1,0,1996,30.1,无记录,106.0,3.818,7.89,0.00,0
1,2,0,1988,27.5,无记录,84.0,-1.000,0.0,14.70,0
2,3,1,1988,36.5,无记录,85.0,7.131,0.0,40.10,1
3,4,1,1992,29.5,无记录,91.0,7.041,0.0,0.00,0
4,5,0,1998,42.0,叔叔或者姑姑有一方患有糖尿病,NaN,7.134,0.0,0.00,1
...,...,...,...,...,...,...,...,...,...,...
9065,3996,0,2007,36.5,无记录,66,3.665,13.28,1.87,-1
9066,3997,1,1990,36.1,叔叔或者姑姑有一方患有糖尿病,60,5.644,8.2,3.34,-1
9067,3998,0,1984,44.6,父母有一方患有糖尿病,83,5.427,32.25,4.61,-1
9068,3999,1,1963,21.9,父母有一方患有糖尿病,89,8.509,0,0.00,-1


In [395]:
pd.isnull(data).any()

编号          False
性别          False
出生年份        False
体重指数        False
糖尿病家族史      False
舒张压          True
口服耐糖量测试     False
胰岛素释放实验     False
肱三头肌皮褶厚度    False
患有糖尿病标识     False
dtype: bool

In [396]:
#将舒张压特征中的缺失值填充为-1
data['舒张压']=data['舒张压'].fillna(-1)

In [397]:
"""
将出生年份换算成年龄
"""
data['年龄']=2022-data['出生年份']  #换成年龄

In [398]:
"""
人体的成人体重指数正常值是在18.5-24之间
低于18.5是体重指数过轻
在24-27之间是体重超重
27以上考虑是肥胖
高于32了就是非常的肥胖。
"""
def BMI(a):
    if containString("无记录", a):
        print('ddd')
    # print(containString("无记录", a))
    if a<18.5:
        return 0
    elif 18.5<=a<=24:
        return 1
    elif 24<a<=27:
        return 2
    elif 27<a<=32:
        return 3
    else:
        return 4

data['BMI']=data['体重指数'].apply(BMI)

In [399]:
def containString(containStr, string):
    """
    :param containStr: 查找包含的字符
    :param string: 所要查找的字符串
    :return: 
    """
    try:
        if isinstance(string, str):
            if string.find(containStr):
                return True
            else:
                return False
        else:
            return False

    except Exception:
        print("error")

In [400]:
#糖尿病家族史
"""
无记录
叔叔或者姑姑有一方患有糖尿病/叔叔或姑姑有一方患有糖尿病
父母有一方患有糖尿病
"""
def FHOD(a):
    if containString("无记录", a):
        return 0
    elif a=='叔叔或者姑姑有一方患有糖尿病' or a=='叔叔或姑姑有一方患有糖尿病':
        return 1
    else:
        return 2

data['糖尿病家族史']=data['糖尿病家族史'].apply(FHOD)

In [401]:
def is_number(num):
        s = str(num)
        if s.count('.') == 1:  # 小数
            new_s = s.split('.')
            left_num = new_s[0]
            right_num = new_s[1]
            if right_num.isdigit():
                if left_num.isdigit():
                    return True
                elif left_num.count('-') == 1 and left_num.startswith('-'):  # 负小数
                    tmp_num = left_num.split('-')[-1]
                    if tmp_num.isdigit():
                        return True
        elif s.count(".") == 0:  # 整数
            if s.isdigit():
                return True
            elif s.count('-') == 1 and s.startswith('-'):  # 负整数
                ss = s.split('-')[-1]
                if ss.isdigit():
                    return True
        return False

In [402]:
float(str('999'))

999.0

In [403]:
"""
舒张压范围为60-90
"""
def DBP(a):
    if not is_number(a):
        return -1
    elif float(a)<60:
        return 0
    elif 60<=float(a)<=90:
        return 1
    elif float(a)>90:
        return 2
    else:
        return a
data['DBP']=data['舒张压'].apply(DBP)

In [404]:
#------------------------------------
#将处理好的特征工程分为训练集和测试集，其中训练集是用来训练模型，测试集用来评估模型准确度
#其中编号和患者是否得糖尿病没有任何联系，属于无关特征予以删除
train=data[data['患有糖尿病标识'] !=-1]
test=data[data['患有糖尿病标识'] ==-1]
train_label=train['患有糖尿病标识']
train=train.drop(['编号','患有糖尿病标识','舒张压','出生年份'],axis=1)
test=test.drop(['编号','患有糖尿病标识','舒张压','出生年份'],axis=1)

In [405]:
def Filter(a):
    if not is_number(a):
        return -1
    else:
        return a

for i in test.columns.tolist():
    test[i]=test[i].apply(Filter)

for i in train.columns.tolist():
    train[i]=train[i].apply(Filter)

In [406]:
#----------------模型训练----------------
model = DecisionTreeClassifier()
model.fit(train, train_label)
y_pre=model.predict(test)
y_pre

array([0, 0, 1, ..., 1, 1, 0], dtype=int64)

In [407]:
len([i+1 for i in range(len(y_pre))])

4000

In [408]:
#----------------结果输出----------------
# result=pd.read_csv('../data/提交示例.csv')
result = {'uuid':0,'label':0}
result['uuid']=[i+1 for i in range(len(y_pre))]
result['label']=y_pre
result = pd.DataFrame(result)
result.to_csv('result-baseline.csv',index=False)